<a href="https://colab.research.google.com/github/mdclary/ODSC_Bayes_Exercise/blob/master/spark_paw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prepare data

Below include your code

In [0]:
!pip install -U -q PyDrive ## you will have install for every colab session

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# 2. Now proxy the Google Drive file to a local file
cancer_data_import = drive.CreateFile({'id':'1FZbQCEHr2Rie4cXSM6Udg0SaWTtPnEHO'})
cancer_data_import.GetContentFile('cancer_data.csv') # 'cancer_data.csv' is the file name that will be accessible in the notebook.b

In [0]:
#Import numpy, pandas and load in the data
import numpy as np
import pandas as pd

df = pd.read_csv("cancer_data.csv")
df.head()

,id,clump-thickness,uniformity-of-cell-size,uniformity-of-cell-shape,marginal-adhesion,single-epithelial-cell-size,bare-nuclei,bland-chromatin,normal-nucleoli,mitoses,class
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2


In [0]:
from sklearn import preprocessing

encoder = preprocessing.LabelEncoder()

for col in df.columns:
    df[col] = encoder.fit_transform(df[col]) # Transform the series so is zero based
    
df.head()

,id,clump-thickness,uniformity-of-cell-size,uniformity-of-cell-shape,marginal-adhesion,single-epithelial-cell-size,bare-nuclei,bland-chromatin,normal-nucleoli,mitoses,class
0,172,4,0,0,0,1,0,2,0,0,0
1,175,4,3,3,4,6,1,2,1,0,0
2,176,2,0,0,0,1,2,2,0,0,0
3,177,5,7,7,0,2,4,2,6,0,0
4,179,3,0,0,2,1,0,2,0,0,0


In [0]:
from sklearn import linear_model as lm
df_keys = list(df.keys())
print(df_keys)
df_keys.remove('id') # Removes id because it is not needed in our calculation
scores = []
for i in df_keys:
  x = df[[i]]
  y = df['class']
  model = lm.LogisticRegression(fit_intercept=True, n_jobs = 1) # Uses logistic regression to fit the model
  model.fit(x,y)
  scores.append([i, model.score(x,y)]) # Appends the model scores into a list.
for i in scores:
  print(i)

['id', 'clump-thickness', 'uniformity-of-cell-size', 'uniformity-of-cell-shape', 'marginal-adhesion', 'single-epithelial-cell-size', 'bare-nuclei', 'bland-chromatin', 'normal-nucleoli', 'mitoses', 'class']
['clump-thickness', 0.8612303290414879]
['uniformity-of-cell-size', 0.927038626609442]
['uniformity-of-cell-shape', 0.9227467811158798]
['marginal-adhesion', 0.8640915593705293]
['single-epithelial-cell-size', 0.8755364806866953]
['bare-nuclei', 0.7138769670958512]
['bland-chromatin', 0.9070100143061517]
['normal-nucleoli', 0.8669527896995708]
['mitoses', 0.7896995708154506]
['class', 1.0]


In [0]:
x = df[['uniformity-of-cell-size', 'uniformity-of-cell-shape', 'bland-chromatin']]
y = df['class']
#Assigns the input vectors and specifies the output column
model = lm.LogisticRegression(fit_intercept=True, n_jobs = 1)
model.fit(x,y)
print('Score:', model.score(x,y)) # Prints out the score of the model


Score: 0.9470672389127325


# Prepare environment and install PySpark

In [0]:
!python -V # must be [3.6, 3.8)
!java -version # must be 1.8

Python 3.6.9
openjdk version "1.8.0_252"
OpenJDK Runtime Environment (build 1.8.0_252-8u252-b09-1~18.04-b09)
OpenJDK 64-Bit Server VM (build 25.252-b09, mixed mode)


In [0]:
!dpkg -l | grep -i openjdk


rc  openjdk-11-jre-headless:amd64          11.0.7+10-2ubuntu2~18.04                          amd64        OpenJDK Java runtime, using Hotspot JIT (headless)
ii  openjdk-8-jre:amd64                    8u252-b09-1~18.04                                 amd64        OpenJDK Java runtime, using Hotspot JIT
ii  openjdk-8-jre-headless:amd64           8u252-b09-1~18.04                                 amd64        OpenJDK Java runtime, using Hotspot JIT (headless)


In [0]:
!sudo apt-get remove openjdk-11-jre -y
!sudo apt-get remove openjdk-11-jre-headless -y
!sudo apt-get install openjdk-8-jre -y
!sudo apt-get install openjdk-8-jre-headless -y


Reading package lists... Done
Building dependency tree       
Reading state information... Done
Package 'openjdk-11-jre' is not installed, so not removed
The following packages were automatically installed and are no longer required:
  cuda-command-line-tools-10-0 cuda-compiler-10-0 cuda-cuobjdump-10-0
  cuda-cupti-10-0 cuda-demo-suite-10-0 cuda-demo-suite-10-1
  cuda-documentation-10-0 cuda-documentation-10-1 cuda-drivers cuda-gdb-10-0
  cuda-gpu-library-advisor-10-0 cuda-libraries-10-0 cuda-memcheck-10-0
  cuda-nsight-compute-10-0 cuda-nsight-compute-10-1 cuda-nsight-systems-10-1
  cuda-nvcc-10-0 cuda-nvdisasm-10-0 cuda-nvml-dev-10-0 cuda-nvprof-10-0
  cuda-nvprune-10-0 cuda-nvtx-10-0 cuda-runtime-10-0 cuda-runtime-10-1
  cuda-samples-10-0 cuda-samples-10-1 dkms freeglut3 freeglut3-dev
  keyboard-configuration libargon2-0 libcap2 libcryptsetup12
  libdevmapper1.02.1 libidn11 libip4tc0 libjansson4 libnvidia-decode-440
  libnvidia-encode-440 libnvidia-extra-440 libnvidia-fbc1-440
  lib

In [0]:
!java -version # must be 1.8

openjdk version "1.8.0_252"
OpenJDK Runtime Environment (build 1.8.0_252-8u252-b09-1~18.04-b09)
OpenJDK 64-Bit Server VM (build 25.252-b09, mixed mode)


In [0]:
!sudo apt-get install g++ liblz4-dev -y


Reading package lists... Done
Building dependency tree       
Reading state information... Done
liblz4-dev is already the newest version (0.0~r131-2ubuntu3).
g++ is already the newest version (4:7.4.0-1ubuntu2.3).
The following packages were automatically installed and are no longer required:
  cuda-command-line-tools-10-0 cuda-compiler-10-0 cuda-cuobjdump-10-0
  cuda-cupti-10-0 cuda-demo-suite-10-0 cuda-demo-suite-10-1
  cuda-documentation-10-0 cuda-documentation-10-1 cuda-drivers cuda-gdb-10-0
  cuda-gpu-library-advisor-10-0 cuda-libraries-10-0 cuda-memcheck-10-0
  cuda-nsight-compute-10-0 cuda-nsight-compute-10-1 cuda-nsight-systems-10-1
  cuda-nvcc-10-0 cuda-nvdisasm-10-0 cuda-nvml-dev-10-0 cuda-nvprof-10-0
  cuda-nvprune-10-0 cuda-nvtx-10-0 cuda-runtime-10-0 cuda-runtime-10-1
  cuda-samples-10-0 cuda-samples-10-1 dkms freeglut3 freeglut3-dev
  keyboard-configuration libargon2-0 libcap2 libcryptsetup12
  libdevmapper1.02.1 libidn11 libip4tc0 libjansson4 libnvidia-decode-440
  libnv

In [0]:
!sudo pip freeze | grep -i albumentations
!sudo pip uninstall albumentations -y

albumentations==0.4.5
Uninstalling albumentations-0.4.5:
  Successfully uninstalled albumentations-0.4.5


In [0]:
!sudo pip install imgaug==0.2.7

  Using cached https://files.pythonhosted.org/packages/af/fc/c56a7da8c23122b7c5325b941850013880a7a93c21dc95e2b1ecd4750108/imgaug-0.2.7-py3-none-any.whl
  Found existing installation: imgaug 0.2.6
    Uninstalling imgaug-0.2.6:
      Successfully uninstalled imgaug-0.2.6


In [0]:
!sudo pip install albumentations

Processing /root/.cache/pip/wheels/f0/a0/61/e50f93165a5ec7e7f5d65064e513239505bc4c06d2289557d3/albumentations-0.4.5-cp36-none-any.whl
Processing /root/.cache/pip/wheels/97/ec/48/0d25896c417b715af6236dbcef8f0bed136a1a5e52972fc6d0/imgaug-0.2.6-cp36-none-any.whl
  Found existing installation: imgaug 0.2.7
    Uninstalling imgaug-0.2.7:
      Successfully uninstalled imgaug-0.2.7


In [0]:
!pip install -U -q pyspark==2.4


# Install Hail

In [0]:
!pip install -U -q hail



# Test hail installation

In [0]:
import hail as hl
import pyspark
hl.init()
mt = hl.balding_nichols_model(n_populations=3, n_samples=50, n_variants=100)
mt.count()

Running on Apache Spark version 2.4.0
SparkUI available at http://f71594f2f984:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.43-d7da1fd14f3c
LOGGING: writing to /content/hail-20200529-1738-0.2.43-d7da1fd14f3c.log
2020-05-29 17:38:32 Hail: INFO: balding_nichols_model: generating genotypes for 3 populations, 50 samples, and 100 variants...


(100, 50)

# Tutorial: GWAS 
https://hail.is/docs/0.2/tutorials/01-genome-wide-association-study.html

In [0]:
import hail as hl
#hl.init()

In [0]:
from hail.plot import show
from pprint import pprint
hl.plot.output_notebook()

## Download and import public 1000 genomes data set

In [0]:
hl.utils.get_1kg('data/')


2020-05-29 17:38:34 Hail: INFO: downloading 1KG VCF ...
  Source: https://storage.googleapis.com/hail-tutorial/1kg.vcf.bgz
2020-05-29 17:38:35 Hail: INFO: importing VCF and writing to matrix table...
2020-05-29 17:38:38 Hail: INFO: Coerced sorted dataset
2020-05-29 17:38:46 Hail: INFO: wrote matrix table with 10961 rows and 284 columns in 16 partitions to data/1kg.mt
2020-05-29 17:38:46 Hail: INFO: downloading 1KG annotations ...
  Source: https://storage.googleapis.com/hail-tutorial/1kg_annotations.txt
2020-05-29 17:38:46 Hail: INFO: downloading Ensembl gene annotations ...
  Source: https://storage.googleapis.com/hail-tutorial/ensembl_gene_annotations.txt
2020-05-29 17:38:47 Hail: INFO: Done!


In [0]:
hl.import_vcf('data/1kg.vcf.bgz').write('data/1kg.mt', overwrite=True)


2020-05-29 17:38:47 Hail: INFO: Coerced sorted dataset
2020-05-29 17:38:53 Hail: INFO: wrote matrix table with 10961 rows and 284 columns in 1 partition to data/1kg.mt


In [0]:
mt = hl.read_matrix_table('data/1kg.mt')


## Get to know the data

In [0]:
mt.rows().select().show(5)


locus,alleles
locus<GRCh37>,array<str>
1:904165,"[""G"",""A""]"
1:909917,"[""G"",""A""]"
1:986963,"[""C"",""T""]"
1:1563691,"[""T"",""G""]"
1:1707740,"[""T"",""G""]"


In [0]:
mt.entry.take(5)


[Struct(GT=Call(alleles=[0, 0], phased=False), AD=[4, 0], DP=4, GQ=12, PL=[0, 12, 147]),
 Struct(GT=Call(alleles=[0, 0], phased=False), AD=[8, 0], DP=8, GQ=24, PL=[0, 24, 315]),
 Struct(GT=Call(alleles=[0, 0], phased=False), AD=[8, 0], DP=8, GQ=23, PL=[0, 23, 230]),
 Struct(GT=Call(alleles=[0, 0], phased=False), AD=[7, 0], DP=7, GQ=21, PL=[0, 21, 270]),
 Struct(GT=Call(alleles=[0, 0], phased=False), AD=[5, 0], DP=5, GQ=15, PL=[0, 15, 205])]

## Add some column fields

In [0]:
table = (hl.import_table('data/1kg_annotations.txt', impute=True)
         .key_by('Sample'))

2020-05-29 17:38:55 Hail: INFO: Reading table to impute column types
2020-05-29 17:38:56 Hail: INFO: Finished type imputation
  Loading column 'Sample' as type 'str' (imputed)
  Loading column 'Population' as type 'str' (imputed)
  Loading column 'SuperPopulation' as type 'str' (imputed)
  Loading column 'isFemale' as type 'bool' (imputed)
  Loading column 'PurpleHair' as type 'bool' (imputed)
  Loading column 'CaffeineConsumption' as type 'int32' (imputed)


In [0]:
table.describe()


----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    'Sample': str 
    'Population': str 
    'SuperPopulation': str 
    'isFemale': bool 
    'PurpleHair': bool 
    'CaffeineConsumption': int32 
----------------------------------------
Key: ['Sample']
----------------------------------------


In [0]:
table.show(width=100)


Sample,Population,SuperPopulation,isFemale,PurpleHair,CaffeineConsumption
str,str,str,bool,bool,int32
"""HG00096""","""GBR""","""EUR""",false,false,4
"""HG00097""","""GBR""","""EUR""",true,true,4
"""HG00098""","""GBR""","""EUR""",false,false,5
"""HG00099""","""GBR""","""EUR""",true,false,4
"""HG00100""","""GBR""","""EUR""",true,false,5
"""HG00101""","""GBR""","""EUR""",false,true,1
"""HG00102""","""GBR""","""EUR""",true,true,6
"""HG00103""","""GBR""","""EUR""",false,true,5
"""HG00104""","""GBR""","""EUR""",true,false,5


In [0]:
print(mt.col.dtype)


struct{s: str}


In [0]:
mt = mt.annotate_cols(pheno = table[mt.s])
mt.col.describe()


--------------------------------------------------------
Type:
        struct {
        s: str, 
        pheno: struct {
            Population: str, 
            SuperPopulation: str, 
            isFemale: bool, 
            PurpleHair: bool, 
            CaffeineConsumption: int32
        }
    }
--------------------------------------------------------
Source:
Index:
    ['column']
--------------------------------------------------------


## Play with query functions and the hail expression language

In [0]:
pprint(table.aggregate(hl.agg.counter(table.SuperPopulation)))


{'AFR': 1018, 'AMR': 535, 'EAS': 617, 'EUR': 669, 'SAS': 661}


In [0]:
pprint(table.aggregate(hl.agg.stats(table.CaffeineConsumption)))


{'max': 10.0,
 'mean': 3.9837142857142855,
 'min': -1.0,
 'n': 3500,
 'stdev': 1.7021055628070711,
 'sum': 13943.0}


In [0]:
table.count()


3500

In [0]:
mt.count_cols()


284

In [0]:
mt.aggregate_cols(hl.agg.counter(mt.pheno.SuperPopulation))


{'AFR': 76, 'AMR': 34, 'EAS': 72, 'EUR': 47, 'SAS': 55}

In [0]:
pprint(mt.aggregate_cols(hl.agg.stats(mt.pheno.CaffeineConsumption)))


{'max': 9.0,
 'mean': 4.415492957746479,
 'min': 0.0,
 'n': 284,
 'stdev': 1.577763427465917,
 'sum': 1254.0}


In [0]:
snp_counts = mt.aggregate_rows(hl.agg.counter(hl.Struct(ref=mt.alleles[0], alt=mt.alleles[1])))
pprint(snp_counts)

{Struct(ref='G', alt='C'): 112,
 Struct(ref='T', alt='A'): 79,
 Struct(ref='A', alt='T'): 76,
 Struct(ref='T', alt='C'): 1879,
 Struct(ref='C', alt='G'): 150,
 Struct(ref='A', alt='G'): 1944,
 Struct(ref='G', alt='A'): 2387,
 Struct(ref='G', alt='T'): 480,
 Struct(ref='A', alt='C'): 454,
 Struct(ref='C', alt='T'): 2436,
 Struct(ref='C', alt='A'): 496,
 Struct(ref='T', alt='G'): 468}


In [0]:
from collections import Counter
counts = Counter(snp_counts)
counts.most_common()

[(Struct(ref='C', alt='T'), 2436),
 (Struct(ref='G', alt='A'), 2387),
 (Struct(ref='A', alt='G'), 1944),
 (Struct(ref='T', alt='C'), 1879),
 (Struct(ref='C', alt='A'), 496),
 (Struct(ref='G', alt='T'), 480),
 (Struct(ref='T', alt='G'), 468),
 (Struct(ref='A', alt='C'), 454),
 (Struct(ref='C', alt='G'), 150),
 (Struct(ref='G', alt='C'), 112),
 (Struct(ref='T', alt='A'), 79),
 (Struct(ref='A', alt='T'), 76)]

In [0]:
p = hl.plot.histogram(mt.DP, range=(0,30), bins=30, title='DP Histogram', legend='DP')
show(p)

# Quality Control

In [0]:
mt.col.describe()


--------------------------------------------------------
Type:
        struct {
        s: str, 
        pheno: struct {
            Population: str, 
            SuperPopulation: str, 
            isFemale: bool, 
            PurpleHair: bool, 
            CaffeineConsumption: int32
        }
    }
--------------------------------------------------------
Source:
Index:
    ['column']
--------------------------------------------------------


In [0]:
mt = hl.sample_qc(mt)
mt.col.describe()


--------------------------------------------------------
Type:
        struct {
        s: str, 
        pheno: struct {
            Population: str, 
            SuperPopulation: str, 
            isFemale: bool, 
            PurpleHair: bool, 
            CaffeineConsumption: int32
        }, 
        sample_qc: struct {
            dp_stats: struct {
                mean: float64, 
                stdev: float64, 
                min: float64, 
                max: float64
            }, 
            gq_stats: struct {
                mean: float64, 
                stdev: float64, 
                min: float64, 
                max: float64
            }, 
            call_rate: float64, 
            n_called: int64, 
            n_not_called: int64, 
            n_filtered: int64, 
            n_hom_ref: int64, 
            n_het: int64, 
            n_hom_var: int64, 
            n_non_ref: int64, 
            n_singleton: int64, 
            n_snp: int64, 
            n_insertio

In [0]:

p = hl.plot.histogram(mt.sample_qc.call_rate, range=(.88,1), legend='Call Rate')
show(p)

In [0]:
p = hl.plot.histogram(mt.sample_qc.gq_stats.mean, range=(10,70), legend='Mean Sample GQ')
show(p)

In [0]:
p = hl.plot.scatter(mt.sample_qc.dp_stats.mean, mt.sample_qc.call_rate, xlabel='Mean DP', ylabel='Call Rate')
show(p)

In [0]:
mt = mt.filter_cols((mt.sample_qc.dp_stats.mean >= 4) & (mt.sample_qc.call_rate >= 0.97))
print('After filter, %d/284 samples remain.' % mt.count_cols())

After filter, 250/284 samples remain.


In [0]:

ab = mt.AD[1] / hl.sum(mt.AD)

filter_condition_ab = ((mt.GT.is_hom_ref() & (ab <= 0.1)) |
                        (mt.GT.is_het() & (ab >= 0.25) & (ab <= 0.75)) |
                        (mt.GT.is_hom_var() & (ab >= 0.9)))

fraction_filtered = mt.aggregate_entries(hl.agg.fraction(~filter_condition_ab))
print(f'Filtering {fraction_filtered * 100:.2f}% entries out of downstream analysis.')
mt = mt.filter_entries(filter_condition_ab)

Filtering 3.64% entries out of downstream analysis.


In [0]:
mt = hl.variant_qc(mt)


In [0]:
mt.row.describe()


--------------------------------------------------------
Type:
        struct {
        locus: locus<GRCh37>, 
        alleles: array<str>, 
        rsid: str, 
        qual: float64, 
        filters: set<str>, 
        info: struct {
            AC: array<int32>, 
            AF: array<float64>, 
            AN: int32, 
            BaseQRankSum: float64, 
            ClippingRankSum: float64, 
            DP: int32, 
            DS: bool, 
            FS: float64, 
            HaplotypeScore: float64, 
            InbreedingCoeff: float64, 
            MLEAC: array<int32>, 
            MLEAF: array<float64>, 
            MQ: float64, 
            MQ0: int32, 
            MQRankSum: float64, 
            QD: float64, 
            ReadPosRankSum: float64, 
            set: str
        }, 
        variant_qc: struct {
            dp_stats: struct {
                mean: float64, 
                stdev: float64, 
                min: float64, 
                max: float64
            }, 

## LETS DO GWAS!

In [0]:
mt = mt.filter_rows(mt.variant_qc.AF[1] > 0.01)
mt = mt.filter_rows(mt.variant_qc.p_value_hwe > 1e-6)
print('Samples: %d  Variants: %d' % (mt.count_cols(), mt.count_rows()))


Samples: 250  Variants: 7849


In [0]:
gwas = hl.linear_regression_rows(y=mt.pheno.CaffeineConsumption,
                                 x=mt.GT.n_alt_alleles(),
                                 covariates=[1.0])
gwas.row.describe()

2020-05-29 17:39:23 Hail: INFO: linear_regression_rows: running on 250 samples for 1 response variable y,
    with input variable x, and 1 additional covariate...


--------------------------------------------------------
Type:
        struct {
        locus: locus<GRCh37>, 
        alleles: array<str>, 
        n: int32, 
        sum_x: float64, 
        y_transpose_x: float64, 
        beta: float64, 
        standard_error: float64, 
        t_stat: float64, 
        p_value: float64
    }
--------------------------------------------------------
Source:
Index:
    ['row']
--------------------------------------------------------


In [0]:
p = hl.plot.manhattan(gwas.p_value)
show(p)

In [0]:
p = hl.plot.qq(gwas.p_value)
show(p)

2020-05-29 17:39:29 Hail: INFO: Ordering unsorted dataset with network shuffle


## Confounded!

In [0]:
eigenvalues, pcs, _ = hl.hwe_normalized_pca(mt.GT)
pprint(eigenvalues)
pcs.show(5, width=100)


2020-05-29 17:39:34 Hail: INFO: hwe_normalized_pca: running PCA using 7841 variants.
2020-05-29 17:39:37 Hail: INFO: pca: running PCA with 10 components...


[18.02377947184679,
 9.988945550363262,
 3.5383122629171173,
 2.657759078373,
 1.596603214765837,
 1.5416611649602368,
 1.5029872248781773,
 1.4720816378531163,
 1.467818848733074,
 1.447783520133495]


s,scores
str,array<float64>
"""HG00096""","[-1.22e-01,-2.81e-01,1.11e-01,-1.28e-01,6.81e-02,-3.72e-03,-2.66e-02,4.99e-03,-9.33e-02,-1.48..."
"""HG00099""","[-1.13e-01,-2.90e-01,1.08e-01,-7.04e-02,4.20e-02,3.33e-02,1.61e-02,-1.15e-03,3.29e-02,2.33e-02]"
"""HG00105""","[-1.08e-01,-2.80e-01,1.03e-01,-1.05e-01,9.40e-02,1.27e-02,3.14e-02,3.08e-02,1.06e-02,-1.93e-02]"
"""HG00118""","[-1.25e-01,-2.98e-01,7.21e-02,-1.07e-01,2.89e-02,8.09e-03,-4.70e-02,-3.32e-02,-2.59e-04,8.49e..."
"""HG00129""","[-1.07e-01,-2.87e-01,9.72e-02,-1.16e-01,1.38e-02,1.87e-02,-8.37e-02,-4.87e-02,3.73e-02,2.11e-02]"


In [0]:
mt = mt.annotate_cols(scores = pcs[mt.s].scores)
p = hl.plot.scatter(mt.scores[0],
                    mt.scores[1],
                    label=mt.pheno.SuperPopulation,
                    title='PCA', xlabel='PC1', ylabel='PC2')
show(p)

In [0]:
gwas = hl.linear_regression_rows(
    y=mt.pheno.CaffeineConsumption,
    x=mt.GT.n_alt_alleles(),
    covariates=[1.0, mt.pheno.isFemale, mt.scores[0], mt.scores[1], mt.scores[2]])

2020-05-29 17:39:50 Hail: INFO: linear_regression_rows: running on 250 samples for 1 response variable y,
    with input variable x, and 5 additional covariates...


In [0]:
p = hl.plot.qq(gwas.p_value)
show(p)

2020-05-29 17:39:52 Hail: INFO: Ordering unsorted dataset with network shuffle


In [0]:
p = hl.plot.manhattan(gwas.p_value)
show(p)

## RARE VARIANT ANALYSIS

In [0]:
entries = mt.entries()
results = (entries.group_by(pop = entries.pheno.SuperPopulation, chromosome = entries.locus.contig)
      .aggregate(n_het = hl.agg.count_where(entries.GT.is_het())))

2020-05-29 17:39:54 Hail: WARN: entries(): Resulting entries table is sorted by '(row_key, col_key)'.
    To preserve row-major matrix table order, first unkey columns with 'key_cols_by()'


In [0]:
results.show()


2020-05-29 17:40:00 Hail: INFO: Ordering unsorted dataset with network shuffle


pop,chromosome,n_het
str,str,int64
"""AFR""","""1""",11276
"""AFR""","""10""",7160
"""AFR""","""11""",6875
"""AFR""","""12""",7048
"""AFR""","""13""",4678
"""AFR""","""14""",4313
"""AFR""","""15""",3904
"""AFR""","""16""",4593
"""AFR""","""17""",3718


In [0]:
entries = entries.annotate(maf_bin = hl.cond(entries.info.AF[0]<0.01, "< 1%",
                             hl.cond(entries.info.AF[0]<0.05, "1%-5%", ">5%")))

results2 = (entries.group_by(af_bin = entries.maf_bin, purple_hair = entries.pheno.PurpleHair)
      .aggregate(mean_gq = hl.agg.stats(entries.GQ).mean,
                 mean_dp = hl.agg.stats(entries.DP).mean))

In [0]:
table = hl.import_table('data/1kg_annotations.txt', impute=True).key_by('Sample')

mt = hl.read_matrix_table('data/1kg.mt')
mt = mt.annotate_cols(pheno = table[mt.s])
mt = hl.sample_qc(mt)
mt = mt.filter_cols((mt.sample_qc.dp_stats.mean >= 4) & (mt.sample_qc.call_rate >= 0.97))
ab = mt.AD[1] / hl.sum(mt.AD)
filter_condition_ab = ((mt.GT.is_hom_ref() & (ab <= 0.1)) |
                        (mt.GT.is_het() & (ab >= 0.25) & (ab <= 0.75)) |
                        (mt.GT.is_hom_var() & (ab >= 0.9)))
mt = mt.filter_entries(filter_condition_ab)
mt = hl.variant_qc(mt)
mt = mt.filter_rows(mt.variant_qc.AF[1] > 0.01)

eigenvalues, pcs, _ = hl.hwe_normalized_pca(mt.GT)

mt = mt.annotate_cols(scores = pcs[mt.s].scores)
gwas = hl.linear_regression_rows(
    y=mt.pheno.CaffeineConsumption,
    x=mt.GT.n_alt_alleles(),
    covariates=[1.0, mt.pheno.isFemale, mt.scores[0], mt.scores[1], mt.scores[2]])

2020-05-29 17:40:00 Hail: INFO: Reading table to impute column types
2020-05-29 17:40:00 Hail: INFO: Finished type imputation
  Loading column 'Sample' as type 'str' (imputed)
  Loading column 'Population' as type 'str' (imputed)
  Loading column 'SuperPopulation' as type 'str' (imputed)
  Loading column 'isFemale' as type 'bool' (imputed)
  Loading column 'PurpleHair' as type 'bool' (imputed)
  Loading column 'CaffeineConsumption' as type 'int32' (imputed)
2020-05-29 17:40:03 Hail: INFO: hwe_normalized_pca: running PCA using 9169 variants.
2020-05-29 17:40:06 Hail: INFO: pca: running PCA with 10 components...
2020-05-29 17:40:14 Hail: INFO: linear_regression_rows: running on 250 samples for 1 response variable y,
    with input variable x, and 5 additional covariates...
